### WeRateDogs Twitter Archive

The goal of this project is to wrangle and analyze the tweets and retweet counts of dog ratings data. WeRateDogs is a great archive, but appears to contain very basic information with regard to the tweet counts, etc..

The missing tweet stats can be completed by making use of the Twitter API where we can query for a given tweet ID and Twitter API will return us a JSON data with other information regarding the tweet.

We are also provided with image predictions of what breed a particular dog belongs to, in a separate file on image_predictions.tsv. These predictions are run using a Neural network.

### Data that we have:

We are already provided with the following datasets:
1. Tweet data from the WerateDogs Twitter Archive - in a .csv file (twitter-archive-enhanced.csv).
2. Predictions from a neural network model that details predictions of dog breed and the score of the predictions. This is from a tsv file saved at "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"


The WeRateDogs Twitter Archive contains observations of 2375 tweets. There are 17 columns in this original dataset. There were several quality issues in this twitter archive and a few tidiness issues found. The one main quality issue drawback I found was the presence of null values in the dataset.

The Neural network predictions dataset is a table of 2075 observations with 12 columns. Each observation is a tweet image - contains the top 3 dog breed prediction, confidence levels and an indicator if the predicted image is a dog.

### Assessment

Below I provide an assessment of the quality issues found in the WeRateDogs Twitter Archive dataset.

- "timestamp" column in werate_dogs_twitter_data is of type str. Converting this to object timestamp can be useful for us while analyzing the data.

- Missing values in retweeted_status_id, retweeted_user_id, retweeted_status_timestamp
- retweeted_status_id, retweeted_user_id are of "np.float64" datatype. It would be useful, if these are integer values.
- dog names Null values: 'name' column contains values such as "None". It appears that the parsing of the names haven't been successful. We can treat these as missing values.
- dog names unclean values: There are values in the 'name' column such as "a", "getting", "actually", etc.. These are not real dog names. These appear to be errors from trying to parse out the names of dogs.
- Some "tweet_id" values result in missing/non-existing tweets and are in some cases wrong tweet IDs. For example: in index #21, Tweet ID: 888202515573088257 doesnt exist. We can however attempt to get the details of this tweet from the expanded_url column.

### Clean

Here are the steps taken to clean up the quality issues found above.

- Convert "timestamp" column to datetime object using pandas to_datetime method.

- retweeted_status_id, retweeted_user_id are of "np.float64" datatype. Convert to int64. Fill null values in these columns   with value -1.

- replace bad data for dog name with data from the tweet text. The value to search for and the value to replace with are found in file missing_dog_names.csv.

- Replaced these unclean values with "None".

- Some "tweet_id" values result in missing/non-existing tweets and are in some cases wrong tweet IDs. For such tweet IDs replace tweet_id with the tweet_id found in the expanded_url column. I will try to achieve this when I query the twitter API below. These values will be retrieved later while querying the twitter API under section "Gathering Data - Getting data from twitter API".

Here is a quick summary of the tidiness issues that were found in the WeRateDogs Twitter Archive dataset.

**Tidiness**

-  Columns doggo, floofer, pupper, puppo are values and not variable names; this violates the 3rd rule of Tidy data. Each observational unit does not form a table.
- Column "source" in the WeRateDogsTwitter Archive data contains two variables, the website reference of the source and the description of the source as a HTML tag.
- Column 'expanded_urls' contains multiple URLs, some of which are repetitions of the same photo URLs. Remove this column from the twitter archive dataset and create a new dataframe for the mediaURLs.

### Assessing the Predictions dataframe

Below is an assessment of the data stored in the **predictions** dataframe.

**Quality**

- Column "tweet_id" is of type object. Each tweet_id is a string, when it will be useful to have tweet_id as an int64 object.

- Columns with predictions, i.e "p1_dog","p2_dog", "p3_dog" are also objects. it will be useful to have these objects as a bool (True/False) data type.

- Columns with predicted confidence values p1_conf, p2_conf, p3_conf are also of type object. It will be useful to convert these into float64 data type.


**Tidiness**

- One issue with the Predictions dataframe is that variable values appear in column names. Columns pn, pn_conf, pn_dog where n=1,2,3 can be melt into a table structure as follows: 'predictionlevelrank', 'PredictedValue', 'confidence value', "is dog" columns in the 'predictions' dataframe.

### Gathering Data from the Twitter API

Now we retrieve twitter data from the Twitter API for each tweet ID listed in the weratedogs twitter Archive. There appears to be some tweet ID's from our original twitter archive csv file, for which the tweets have been deleted. Such tweets do not carry other info, such as # of Retweets, likes etc.. I have collected such tweets in file "unauthorized_tweet_ids.csv". For example, Tweet ID 680055455951884288 appears to be deleted or not existing anymore.

There also appear to be certain Tweet ID's that do not contain a valid ID, however, the retweet ID/the tweet ID found in the media URL is valid and turns out that these tweets are not really retweets and are tweeted by User @dog_rates.
For example, tweet ID: 770743923962707968 doesnt exist. URL: https://twitter.com/dog_rates/status/770743923962707968 doesnt exist. However, the media URLs associated with this row in archive returns a valid tweet, which is at the website:
https://twitter.com/dog_rates/status/739238157791694849/video/1 , where we see this dog blowing bubbles :) 
The below code collects the JSON object pertaining to the tweet ID found from the Media URL/ retweeted ID.

Overall there are 5 tweet ID's in the archive that are unauthorized/deleted.

The JSON tweet data from each tweet was output to a file Tweet-json.txt. Each line of thist txt file is a string that is of a JSON dictionary notation containing information such as # retweets, favorite counts, etc..
Now using this txt file, I create a dataframe whose columns correspond to the tweet ID of a tweet, Retweet Count, Favorite Count, mediaURLs associated, retweeted (yes/no). This dataframe was merged with our original twitter archive master dataframe and the resulting dataframe was written to a .csv file.

### Outputs

``master_data.csv``: This contains the output of the master dataframe obtained by merging the original twitter archive dataframe and the dataframe obtained from retrieving data from the Twitter API.<br>
``predictions_cleaned.csv``: The cleaned Predictions data obtained from the .tsv file. <br>
``mediaURLs.csv``: This is the expanded_urls column, that violated the Tidy Data principle. This file contains the mapping of a tweet ID to the URLs that it contains. <br>